In [ ]:
!nvidia-smi

Wed Jul  5 07:03:15 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/Python/Bearing Faults Project'

Mounted at /content/drive
/content/drive/MyDrive/Python/Bearing Faults Project


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from random import randint
import utils
import time
#import utils_lenet5
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
bs = 1

cuda


#preprocessing

In [ ]:
import re
def Get_Label(file_name):
    fault = re.search(r'([A-Za-z]+)', file_name).group()
    return fault
faults_map = {
    'N': 0,
    'I': 1,
    'O': 2,
    'B': 3,
    'IO': 4,
    'IB': 5,
    'OB': 6,
}

In [ ]:
import os
import scipy.io
import scipy.io as sio
import torch

x_train = torch.rand(21000, 2048, 2)
y_train = torch.zeros(21000)
segment_length = 2048
overlap = 0.75
folder_path = "/content/drive/MyDrive/Python/Bearing Faults Project/HUST bearing dataset"
file_list = os.listdir(folder_path)
mat_files = [f for f in file_list if f.endswith('.mat')]
idx = 0
count_file = 0
for file_name in mat_files:
    file_path = os.path.join(folder_path, file_name)
    data = scipy.io.loadmat(file_path)
    bearing_data = data['data']
    bearing_data = torch.from_numpy(bearing_data)
    #y_train[idx] = faults_map[Get_Label(file_name)]
    segments = []
    total_length = bearing_data.shape[0]
    stride = int(segment_length * (1 - overlap))
    check_label = faults_map[Get_Label(file_name)]
    if check_label == 3:
        num_segments = 250
    elif check_label == 5:
        num_segments = 250
    else:
        num_segments = 200
    for i in range(num_segments):
        y_train[idx] = faults_map[Get_Label(file_name)]
        start = i * stride
        end = start + segment_length
        segment = bearing_data[start:end]
        segment = torch.cat((segment, segment), dim=1)
        x_train[idx] = segment
        idx += 1


In [ ]:
from sklearn.model_selection import train_test_split
train_data, test_data, train_label, test_label = train_test_split(x_train, y_train, test_size=0.2, random_state=42)
print(train_data.shape)
print(train_label.shape)
print(test_data.shape)
print(test_label.shape)

torch.Size([16800, 2048, 2])
torch.Size([16800])
torch.Size([4200, 2048, 2])
torch.Size([4200])


In [ ]:
train_data = train_data.reshape(16800, 4096)
test_data = test_data.reshape(4200, 4096)
#
train_data_bf_spec = train_data
test_data_bf_spec = test_data

In [ ]:
#Transform to spectrogram
import librosa
import numpy as np
import cv2

#change to spectrogram TRAIN_DATA
spectrograms = []
for i in range(train_data.shape[0]):
    signal = train_data[i, :]
    signal = np.array(signal)
    spectrogram = librosa.stft(signal, n_fft=512, hop_length=512)
    spectrogram = np.abs(spectrogram)**2
    log_spectrogram = librosa.power_to_db(spectrogram)
    log_spectrogram = cv2.resize(log_spectrogram, (64, 64))
    spectrograms.append(log_spectrogram)
train_data = np.stack(spectrograms)


#change to spectrogram TEST_DATA

spectrograms_testdata = []
for i in range(test_data.shape[0]):
    signal = test_data[i, :]
    signal = np.array(signal)
    spectrogram = librosa.stft(signal, n_fft=512, hop_length=512)
    spectrogram = np.abs(spectrogram)**2
    log_spectrogram = librosa.power_to_db(spectrogram)
    log_spectrogram = cv2.resize(log_spectrogram, (64, 64))
    spectrograms_testdata.append(log_spectrogram)
test_data = np.stack(spectrograms_testdata)

#to tensor
train_data = train_data.astype(np.float32)
test_data = test_data.astype(np.float32)
train_data = torch.from_numpy(train_data)
test_data = torch.from_numpy(test_data)

train_data = train_data.unsqueeze(dim = 1)
test_data = test_data.unsqueeze(dim = 1)

print(train_data.shape)
print(test_data.shape)
print(train_label.shape)

torch.Size([16800, 1, 64, 64])
torch.Size([4200, 1, 64, 64])
torch.Size([16800])


#Split the set of traindata

In [ ]:
#-----Test with limited data-------#
train_data_1, train_data_99, train_label_1, train_label_99 = train_test_split(train_data, train_label, test_size=0.99, random_state=42)
print(train_data_1.shape)
print(train_label_1.shape)
print(train_data_99.shape)
print(train_label_99.shape)

torch.Size([168, 1, 64, 64])
torch.Size([168])
torch.Size([16632, 1, 64, 64])
torch.Size([16632])


In [ ]:
#Test with 700 samples
test_data_1, test_data_99, test_label_1, test_label_99 = train_test_split(test_data, test_label, test_size=5/6, random_state=42, stratify=test_label)

print(test_data_1.shape)
print(test_label_1.shape)
print(test_data_99.shape)
print(test_label_99.shape)

torch.Size([700, 1, 64, 64])
torch.Size([700])
torch.Size([3500, 1, 64, 64])
torch.Size([3500])


#Covmnet


In [ ]:
from torch.utils.data import Dataset


class CustomDataset(Dataset):
    def __init__(self, train_data, train_label, episode_num=1000, way_num=7, shot_num=1, query_num=1):
        self.train_data = train_data
        self.train_label = train_label
        self.episode_num = episode_num
        self.way_num = way_num
        self.shot_num = shot_num
        self.query_num = query_num

    def __len__(self):
        return self.episode_num

    def __getitem__(self, index):
        query_images = []
        query_targets = []
        support_images = []
        support_targets = []

        label_indices = torch.randperm(len(self.train_label)).cuda()
        train_label_gpu = self.train_label.cuda()
        train_data_gpu = self.train_data.cuda()

        for label_num in range(self.way_num):
            support_idxs = torch.nonzero(train_label_gpu[label_indices] == label_num, as_tuple=False).flatten()
            support_idxs = support_idxs[:self.shot_num]
            support_data = train_data_gpu[label_indices][support_idxs]

            query_idxs = torch.nonzero(train_label_gpu[label_indices] == label_num, as_tuple=False).flatten()
            query_idxs = query_idxs[~torch.isin(query_idxs, support_idxs)][:self.query_num]
            query_data = train_data_gpu[label_indices][query_idxs]
            query_data_targets = train_label_gpu[label_indices][query_idxs]

            query_images.append(query_data)
            query_targets.append(query_data_targets)
            support_images.append(support_data)
            support_targets.append(torch.full((self.shot_num,), label_num).cuda())

        query_images = torch.cat(query_images, dim=0)
        query_targets = torch.cat(query_targets, dim=0)
        support_images = torch.cat(support_images, dim=0)
        support_targets = torch.cat(support_targets, dim=0)

        return query_images, query_targets, support_images, support_targets



'''
  way_num = num_classes, shot_num = number samples per class

'''
train_dataset = CustomDataset(train_data_1, train_label_1, episode_num=100, way_num=7, shot_num=1, query_num=1)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=bs, shuffle=True)
test_dataset = CustomDataset(test_data_1, test_label_1, episode_num=100, way_num=7, shot_num=1, query_num=1)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=True)

###Check Custom dataset

###Function

In [ ]:
def Cal_Accuracy(loader, net):
    true_label = 0
    num_batches = 0
    for query_images, query_targets, support_images, support_targets in loader:
        q = query_images.permute(1, 0, 2, 3, 4).to(device)
        s = support_images.permute(1, 0, 2, 3, 4).to(device)
        targets = query_targets.to(device)
        targets = targets.permute(1,0)
        for i in range(len(q)):
            scores = net(q[i], s).float()
            target = targets[i].long()
            true_label += 1 if torch.argmax(scores) == target else 0
            num_batches += 1
    return true_label/num_batches

def Convert_For_5shots(support_images, support_targets):
    support_targets = support_targets.cpu()
    labels = torch.unique(support_targets)
    new_support_images = []
    for label in labels:
        label_images = support_images[:, support_targets[0] == label]
        padded_label_images = torch.zeros((7, 1, 64, 64), dtype=label_images.dtype)
        padded_label_images[:label_images.shape[1]] = label_images.squeeze(0)
        new_support_images.append(padded_label_images.to(device))
    return new_support_images

###Model

####Preparation

In [ ]:
import torch
import torch.nn as nn
#-----------------------Convmixer-----------------------------------------------#
class Residual(nn.Module):
    def __init__(self, fn):
        super().__init__()
        self.fn = fn

    def forward(self, x):
        return self.fn(x) + x

def ConvMixer(inplane = 1, dim = 64, depth=1, kernel_size=64, patch_size=4):
    return nn.Sequential(
        nn.Conv2d(inplane , dim, kernel_size=patch_size, stride=patch_size),
        nn.GELU(),
        nn.BatchNorm2d(dim),
        *[nn.Sequential(
            Residual(nn.Sequential(
                nn.Conv2d(dim, dim, (1, kernel_size), groups=dim, padding="same"),
                nn.GELU(),
                nn.BatchNorm2d(dim),
                nn.Conv2d(dim, dim, (kernel_size, 1), groups=dim, padding="same"),
                nn.GELU(),
                nn.BatchNorm2d(dim)
            )),
            nn.Conv2d(dim, dim, kernel_size=1),
            nn.GELU(),
            nn.BatchNorm2d(dim)
        ) for i in range(depth)]
    )

#--------Multi-head attention Module-----------------------------------------------------------------------#
class MHSA(nn.Module):
    def __init__(self, n_dims = 64, width=16, height=16, heads=4):
        super(MHSA, self).__init__()
        self.heads = heads

        self.query = nn.Conv2d(n_dims, n_dims, kernel_size=1)
        self.key = nn.Conv2d(n_dims, n_dims, kernel_size=1)
        self.value = nn.Conv2d(n_dims, n_dims, kernel_size=1)

        self.rel_h = nn.Parameter(torch.randn([1, heads, n_dims // heads, 1, height]), requires_grad=True)
        self.rel_w = nn.Parameter(torch.randn([1, heads, n_dims // heads, width, 1]), requires_grad=True)

        self.softmax = nn.Softmax(dim=-1)
        self.norm = nn.LayerNorm([n_dims, width, height])

    def forward(self, x):
        n_batch, C, width, height = x.size()
        q = self.query(x).view(n_batch, self.heads, C // self.heads, -1)
        k = self.key(x).view(n_batch, self.heads, C // self.heads, -1)
        v = self.value(x).view(n_batch, self.heads, C // self.heads, -1)

        content_content = torch.matmul(q.permute(0, 1, 3, 2), k)

        content_position = (self.rel_h + self.rel_w).view(1, self.heads, C // self.heads, -1).permute(0, 1, 3, 2)
        content_position = torch.matmul(content_position, q)

        energy = content_content + content_position
        attention = self.softmax(energy)

        out = torch.matmul(v, attention.permute(0, 1, 3, 2))
        out = out.view(n_batch, C, width, height)
        out = self.norm(out) + x
        return out
'''
test = torch.rand(1, 1, 64, 64)
mixer = ConvMixer()
model = MHSA()
output = model(mixer(test))
output.shape
'''

'\ntest = torch.rand(1, 1, 64, 64)\nmixer = ConvMixer()\nmodel = MHSA()\noutput = model(mixer(test))\noutput.shape\n'

####Proposed

In [ ]:
import functools

class CovarianceNet_64(nn.Module):
    def __init__(self, norm_layer=nn.BatchNorm2d, num_classes=7):
        super(CovarianceNet_64, self).__init__()

        if type(norm_layer) == functools.partial:
            use_bias = norm_layer.func == nn.InstanceNorm2d
        else:
            use_bias = norm_layer == nn.InstanceNorm2d

        self.mixer = ConvMixer()
        self.MHSA = MHSA()

        self.covariance = CovaBlock()

        self.classifier = nn.Sequential(
            nn.LeakyReLU(0.2, True),
            nn.Dropout(),
            nn.Conv1d(1, 1, kernel_size=256, stride=256, bias=use_bias),
        )


    def forward(self, input1, input2):
        #q = self.features(input1)
        q = self.MHSA(self.mixer(input1))
        S = []
        for i in range(len(input2)):
            features = self.mixer(input2[i])
            S.append(self.MHSA(features))
        x = self.covariance(q, S)
        x = self.classifier(x.view(x.size(0), 1, -1))
        x = x.squeeze(1)


        return x


class CovaBlock(nn.Module):
    def __init__(self):
        super(CovaBlock, self).__init__()

    def cal_covariance(self, input):
        CovaMatrix_list = []
        for i in range(len(input)):
            support_set_sam = input[i]
            B, C, h, w = support_set_sam.size()

            support_set_sam = support_set_sam.permute(1, 0, 2, 3)
            support_set_sam = support_set_sam.contiguous().view(C, -1)
            mean_support = torch.mean(support_set_sam, 1, True)
            support_set_sam = support_set_sam - mean_support

            covariance_matrix = support_set_sam @ torch.transpose(support_set_sam, 0, 1)
            covariance_matrix = torch.div(covariance_matrix, h * w * B - 1)
            CovaMatrix_list.append(covariance_matrix)
        return CovaMatrix_list

    def cal_similarity(self, input, CovaMatrix_list):
        B, C, h, w = input.size()
        Cova_Sim = []

        for i in range(B):
            query_sam = input[i]
            query_sam = query_sam.view(C, -1)
            query_sam_norm = torch.norm(query_sam, 2, 1, True)
            query_sam = query_sam / query_sam_norm

            if torch.cuda.is_available():
                mea_sim = torch.zeros(1, len(CovaMatrix_list) * h * w).cuda()
            else:
                mea_sim = torch.zeros(1, len(CovaMatrix_list) * h * w)
            for j in range(len(CovaMatrix_list)):
                temp_dis = torch.transpose(query_sam, 0, 1) @ CovaMatrix_list[j] @ query_sam
                mea_sim[0, j * h * w:(j + 1) * h * w] = temp_dis.diag()

            Cova_Sim.append(mea_sim.view(1, -1))

        Cova_Sim = torch.cat(Cova_Sim, 0)

        return Cova_Sim

    def forward(self, x1, x2):
        CovaMatrix_list = self.cal_covariance(x2)
        Cova_Sim = self.cal_similarity(x1, CovaMatrix_list)

        return Cova_Sim


In [ ]:
model = CovarianceNet_64().cuda()
x1 = torch.rand(7,5,1,64,64).cuda()
x2 = torch.rand(1,1,64,64).cuda()
model(x2,x1).shape


torch.Size([1, 7])

###Training


In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(device)

cuda


In [ ]:
#device= torch.device("cuda")
net = CovarianceNet_64()
net.to(device)
lr = 5e-3
criterion = nn.CrossEntropyLoss()


In [ ]:
optimizer = optim.Adam(net.parameters(), lr=lr)
scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
criterion.to(device)
full_loss = []
pred_acc = 0

for epoch in range(1, 60):
    running_loss = 0
    num_batches = 0
    optimizer.zero_grad()

    for query_images, query_targets, support_images, support_targets in train_dataloader:
        '''
        input: query_images (size: 16 (batchsize) x 7(catagories) x 1(chanel) x 64(H) x 64(W))
              support_images (size: 16 (batchsize) x 7(catagories) x 1(chanel) x 64(H) x 64(W)) (One shot)

              each query_images will compare the covariance with each sample in the support set
        '''
        q = query_images.permute(1, 0, 2, 3, 4).to(device)
        s = support_images.permute(1, 0, 2, 3, 4).to(device)
        targets = query_targets.to(device)
        targets = targets.permute(1,0)
        for i in range(len(q)):
            scores = net(q[i], s).float()
            target = targets[i].long()
            loss = criterion(scores, target)
            loss.backward()
            running_loss += loss.detach().item()
            num_batches += 1

        optimizer.step()
        optimizer.zero_grad()

    scheduler.step()

    with torch.no_grad():
        #-----On training set-----------#
        total_loss = running_loss / num_batches
        full_loss.append(total_loss)
        print('epoch =', epoch, '\t lr =', optimizer.param_groups[0]['lr'], '\t loss =', total_loss)
        print('------------Testing on the test set-------------')
        acc = Cal_Accuracy(test_dataloader, net)
        print(f'Accuracy on the test set: {acc:.4f}')
        if acc > pred_acc:
            pred_acc = acc
            model_name = f'Covamnet_1shot_{acc:.4f}_1%.pth'
            torch.save(net, '/content/drive/MyDrive/Python/Bearing Faults Project/' + model_name)
            print(f'=> Save the best model with accuracy is: {acc:.4f}')
        print("-------------------------------------------------------------------------------------------")

    torch.cuda.empty_cache()


#Validation

###1% dataset

In [ ]:
for query_images, query_targets, support_images, support_targets in train_dataset:
    Support_to_test = support_images
    Support_to_test = Support_to_test.unsqueeze(dim = 1)
    print(Support_to_test.shape)
    break

torch.Size([7, 1, 1, 64, 64])


In [ ]:
model = CovarianceNet_64().cuda()
model = torch.load('Covamnet_1shot_0.5514_1%.pth')
test = test_data_1.unsqueeze(dim = 1).cuda()
Support_to_test = Support_to_test.cuda()
true = 0
total = 0
for index in range(len(test)):
    true += 1 if torch.argmax(model(test[index], Support_to_test)) == test_label_1[index] else  0
    total +=1
print(true/total)

0.6657142857142857


###10% dataset

In [ ]:
model = torch.load('Covamnet_1shot_0.9671_10%data.pth')
test = test_data.unsqueeze(dim = 1).cuda()
Support_to_test = Support_to_test.cuda()
true = 0
total = 0
for index in range(len(test)):
    true += 1 if torch.argmax(model(test[index], Support_to_test)) == test_label[index] else  0
    total +=1
print(true/total)

0.9614285714285714
